In [ ]:
# Librerias 

import pandas as pd
import pyperclip as ctrl
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert
from selenium.common.exceptions import TimeoutException
import time
from datetime import datetime

# Origen de información

FamSalidas = pd.read_csv('Archivos\mfamilias.csv', delimiter = ';', index_col = 'Hogar')['ID'].tolist()
BenSalidos = pd.read_csv('Archivos\salidos.csv', delimiter = ';', index_col = 'Ben')['ID'].tolist()
bd = pd.read_csv('Archivos\caminos.csv', delimiter = ";", index_col = "ID",)
dig = pd.read_csv('Archivos\BD.csv', delimiter = ";", index_col = "ID",)

# Funciones
    
def login(): # Iniciar sesion
    driver.find_element('xpath','//*[@id="txtUsername"]').send_keys("jenielwtf@gmail.com")
    driver.find_element('xpath','//*[@id="txtPassword"]').send_keys("1qazxsw2")
    driver.find_element('xpath','//*[@id="btnLogin"]').click()    
    
def firma(): #Firma y plan de caso
    Select(driver.find_element('xpath',bd.iloc[5,1])).select_by_visible_text("Si")
    firma = driver.find_element('xpath',bd.iloc[6,1])
    Select(firma).select_by_value(dig.iloc[col,5])
    time.sleep(1)
    Select(driver.find_element('xpath',bd.iloc[7,1])).select_by_visible_text("Si")
    
def visita(): # Motivo y lugar de visita
    if dig.iloc[col,2] == "Visita regular":  # Motivo de la Visita
        Select(driver.find_element('xpath',bd.iloc[3,1])).select_by_value("1") 
    elif dig.iloc[col,2] == "Seguimiento":
        Select(driver.find_element('xpath',bd.iloc[3,1])).select_by_value("2")
    elif dig.iloc[col,2] == "Monitoreo":
        Select(driver.find_element('xpath',bd.iloc[3,1])).select_by_value("3")
            
    if dig.iloc[col,3] == "SAI":  # Lugar del servicio
        Select(driver.find_element('xpath',bd.iloc[4,1])).select_by_value("1")
    elif dig.iloc[col,3] == "HOGAR":
        Select(driver.find_element('xpath',bd.iloc[4,1])).select_by_value("2")
        
def beneficiario(): # Hace un recorrido entre los beneficiarios y le va marcando su servicio 
    coun = 1
    miembros = driver.find_element('xpath','//*[@id="MainContent_cbohhMember"]')
    limit = len(miembros.text.split("\n"))-1
    
    while coun < limit:      
        miembros = driver.find_element('xpath','//*[@id="MainContent_cbohhMember"]')
        Select(miembros).select_by_index(coun)     
        
        try:
            WebDriverWait(driver,3).until(EC.alert_is_present())
            alert = driver.switch_to.alert
            alert.accept()
            print("Beneficiario con 21 años")
        except TimeoutException:
            miembros = driver.find_element('xpath','//*[@id="MainContent_cbohhMember"]')
            Select(miembros).select_by_index(coun)
            flag = miembros.get_attribute('value') in BenSalidos              
            if flag == False:     
                edad = driver.find_element('xpath','//*[@id="MainContent_txtAge"]').get_attribute("value")        
                escuela = driver.find_element('xpath','//*[@id="MainContent_cboEnrolledInSchool"]')
                actividad = driver.find_element('xpath','//*[@id="MainContent_cboEnrolledEconomicActivity"]')
                if int(edad) > 17 and int(edad) < 21:
                    Select(escuela).select_by_index(1)
                    Select(actividad).select_by_index(2)
                else:
                    Select(escuela).select_by_index(3)
                    Select(actividad).select_by_index(3)
                driver.find_element('xpath','//*[@id="MainContent_mainPanal"]/a[7]').click() # Aquí empieza a servir
                Select(driver.find_element('xpath','//*[@id="MainContent_cboFoodDeliveryservice"]')).select_by_index(1)
                Select(driver.find_element('xpath','//*[@id="MainContent_cboFoodDeliveryservice_dnr"]')).select_by_index(9)
                time.sleep(0.5)
                driver.find_element('xpath','//*[@id="MainContent_btnsave"]').click() # Guarda el servicio
                wait.until(EC.alert_is_present()).accept() # Espera
            else:
                print('Beneficiario salido')      
        
        coun += 1
    
    print('Servicio digitado')
    


In [2]:
# Abrir navegador e iniciar sesion en el MIS

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://pactbrmis.org/Account/Login.aspx")
login()
wait = WebDriverWait(driver,4)

[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:07<00:00, 1.01MB/s]


In [4]:
beneficiario()

Servicio digitado


In [ ]:
# Inicio Para digitar servicios que aplican a toda la familia 

col = 16 # Realmente es fila
while col < 18:

    # Iniciar servicios 

    driver.get("https://pactbrmis.org/DataEntry/service_delivery.aspx?tokenID=&action=")
    #time.sleep(1)

    # Hogar

    driver.find_element('xpath',bd.iloc[0,1]).click()
    driver.find_element('xpath', bd.iloc[1,1]).send_keys(dig.iloc[col,0], Keys.ENTER)

    # Fecha
    
    set_fecha = datetime.strptime(dig.iloc[col,1], '%d/%m/%Y')
    fecha = set_fecha.strftime('%d/%m/%Y')
    ctrl.copy(fecha)
    driver.find_element('xpath',bd.iloc[2,1]).send_keys(Keys.CONTROL, 'v', Keys.ENTER)

    visita()
    firma()
    driver.find_element('xpath','//*[@id="MainContent_btnsaveMain"]').click()
    wait.until(EC.alert_is_present()).accept()
    beneficiario()
    print('Siguiente servicio')    
    col += 1
    
print('Se digitaron todos los servicios')

In [ ]:
set_fecha = datetime.strptime(dig.iloc[col,1], '%d/%m/%Y')
fecha = set_fecha.strftime('%d/%m/%Y')

print(fecha)

In [ ]:
driver.quit()

In [ ]:
n = int(input())
parte1 = ((-1) ** (n + 1))
parte2 = ((n ** 2) / ((3 * n) - 1))
print(str(parte1 * parte2))